## 고도차이와 위치 데이터를 기반으로 경사'각' 계산

In [1]:
import pandas as pd

In [2]:
data_url = './make_file/자동차_도로_고도차이_포함_데이터.csv'

In [3]:
data = pd.read_csv(data_url)
data.head(1)

,도로명,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,도로_길이,터널 여부,교량 여부,서비스 도로 여부,도로 ID,속도 제한,차선 수,일방통행 여부,교차로 여부,시작점_고도,종료점_고도,고도의_차이
0,"['광장앞지하차도', '광장로']",3차로,37.545509,127.097772,37.546747,127.102031,37.546128,127.099902,430.542185,yes,NaN,NaN,"[175320385, 175320419, 175320429]",NaN,NaN,NaN,NaN,39.830399,21.125259,18.705139


In [4]:
data.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '터널 여부', '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한',
       '차선 수', '일방통행 여부', '교차로 여부', '시작점_고도', '종료점_고도', '고도의_차이'],
      dtype='object')

In [5]:
# 길이 기반
length = data['도로_길이'].to_list()
different_alt = data['고도의_차이'].to_list()

In [6]:
# 불러온 값의 길이 확인
len_1 = len(length)
len_2 = len(different_alt)
max_len = None


if len_1 == len_2 :
    max_len = len_1 
    print(f'length 길이 : {len_1}')
    print(f'different_alt 길이 : {len_2}')
else :
    max_len = 0

length 길이 : 6232
different_alt 길이 : 6232


In [7]:
import numpy as np

In [8]:
# 결과 저장 리스트
road_slope = []

In [9]:
for i in range(max_len) : 
    theta = np.degrees(np.arctan2(different_alt[i], length[i]))
    road_slope.append(theta)

In [10]:
print(f'변환한 road_slope의 길이 : {len(road_slope)}')

변환한 road_slope의 길이 : 6232


In [11]:
new_add_df = pd.DataFrame({
    '경사각' : road_slope
})

In [12]:
data['경사각'] = new_add_df

In [13]:
data.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '터널 여부', '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한',
       '차선 수', '일방통행 여부', '교차로 여부', '시작점_고도', '종료점_고도', '고도의_차이', '경사각'],
      dtype='object')

In [14]:
new_order = [
    '도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도',
    '중앙점_위도', '중앙점_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각', '도로_길이'
] + [col for col in data.columns if col not in [
    '도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도',
    '중앙점_위도', '중앙점_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각', '도로_길이'
]]

# 컬럼 순서 재배열
data = data[new_order]


In [15]:
data.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각', '도로_길이', '터널 여부',
       '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한', '차선 수', '일방통행 여부', '교차로 여부'],
      dtype='object')

In [16]:
data.head(5)

,도로명,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,시작점_고도,종료점_고도,...,경사각,도로_길이,터널 여부,교량 여부,서비스 도로 여부,도로 ID,속도 제한,차선 수,일방통행 여부,교차로 여부
0,"['광장앞지하차도', '광장로']",3차로,37.545509,127.097772,37.546747,127.102031,37.546128,127.099902,39.830399,21.125259,...,2.487682,430.542185,yes,NaN,NaN,"[175320385, 175320419, 175320429]",NaN,NaN,NaN,NaN
1,워커힐로,주거지 도로,37.545509,127.097772,37.546539,127.098053,37.546024,127.097913,39.830399,50.531334,...,5.209767,117.361840,NaN,NaN,NaN,516647748,NaN,NaN,NaN,NaN
2,워커힐로,주거지 도로,37.545509,127.097772,37.543981,127.097208,37.544745,127.097490,39.830399,30.904158,...,2.883715,177.203342,NaN,NaN,NaN,516647752,NaN,NaN,NaN,NaN
3,자양번영로,3차로,37.533200,127.075680,37.533816,127.075863,37.533508,127.075772,15.159596,15.783295,...,0.507954,70.349590,NaN,NaN,NaN,37401581,30.0,4.0,NaN,NaN
4,자양번영로,3차로,37.533200,127.075680,37.532597,127.075517,37.532898,127.075599,15.159596,14.775410,...,0.321100,68.552097,NaN,NaN,NaN,516723466,NaN,NaN,NaN,NaN


In [17]:
try :
    save_file_name = './make_file/(최종)자동차_도로_고도차이(경사각)_포함_데이터.csv'
    data.to_csv(save_file_name, encoding='euc-kr')
    print(f'{save_file_name} 파일 저장 성공')
except OSError as e: 
    print(e)

./make_file/자동차_도로_고도차이(경사각)_포함_데이터.csv 파일 저장 성공
